# I. Imports

In [8]:
# External libraries

import numpy as np
import matplotlib.pyplot as plt

# Internal imports

from src.data import X, y, d
from src.aggregationFunctions import uniform_agg, gaussian_agg
from src.functions import *
from src.samplers import cmc
from src.variationalObjective import variational_obj
from src.gradientDescent import gradient_descent_W
from src.samplers import gibbs_sampler

# II. Sample from data using CMC

In [6]:
ncores = 10
W = np.array([np.eye(d) for k in range(ncores)])
betas = cmc(
    X = X,
    y = y,
    ncores=ncores,
    nsim=100,
    burnin=10
)

# III. Gradient descent

In [7]:
lr = 1
niter = 10
tol = 1e-6
sigma = 1

K,nsamples,d=betas.shape
S = np.array([np.mean([np.outer(betas[k,i],betas[k,i]) for i in range(nsamples)],axis=0) for k in range(K)])
mu = np.mean(betas,axis=1)
W = np.array([np.eye(d)/K for _ in range(K)]) #usual initialisation

W = gradient_descent_W(
    W_0 = W,
    S = S,
    mu = mu,
    betas = betas, 
    X = X,
    y = y,
    learning_rate = lr,
    niter = niter,
    tolerance = tol
)

iter: 0


KeyboardInterrupt: 

In [6]:
betas_vcmc=np.sum([(W/np.sum(W,axis=0))[k] @np.mean(betas[k],axis=0) for k in range(K)],axis=0)
betas_vcmc/betas_vcmc[0]

array([ 1.        , -0.15523924, 79.12822357,  1.14072088,  0.96038183])

# Results

In [9]:
betas=gibbs_sampler(X,y,nsim=4000,burnin=500)

In [11]:
map_estimator=betas.mean(axis=0)
map_estimator/=map_estimator[0]
map_estimator

array([ 1.        , -0.14046254,  0.03701395, -0.34374295, -0.87787349])

In [12]:
ncores_list=[5,10,25,50,100]
err_uni=[]
err_gauss=[]
for ncores in ncores_list:
    sub_thetas=cmc(X,y,ncores=ncores,nsim=2000,burnin=500)
    theta_uni=uniform_agg(sub_thetas)
    theta_gauss=gaussian_agg(sub_thetas)
    theta_uni/=theta_uni[0]
    theta_gauss/=theta_gauss[0]
    err_uni.append(np.linalg.norm(map_estimator-theta_uni,ord=1)/np.linalg.norm(map_estimator,ord=1))
    err_gauss.append(np.linalg.norm(map_estimator-theta_gauss,ord=1)/np.linalg.norm(map_estimator,ord=1))

In [ ]:
X_axis = np.arange(len(ncores_list))
plt.bar(X_axis - 0.2,err_uni, 0.4, label = 'Uniform')
plt.bar(X_axis + 0.2,err_gauss, 0.4, label = 'Gaussian')
plt.xticks(X_axis, ncores_list)
plt.xlabel("Number of cores")
plt.ylabel("Error")
plt.legend()
plt.show()